In [1]:
import os
import toml
from dotenv import load_dotenv
from src.document_loader import DocumentLoader
from src.embeddings import Embeddings
from src.vector_store import VectorStore
from src.retriever import Retriever
from src.llm import LLM
from src.rag_agent import RAGAgent

[nltk_data] Downloading package punkt to /home/chollar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
load_dotenv()

env = os.getenv('ENV', 'development')

# Load configuration
with open(f'config/config.{env}.toml', 'r') as file:
    config = toml.load(file)
    
model_id = config['model']['id']
document_directory = config['documents']['directory']
embedding_directory = os.path.join(document_directory, "embeddings")
    
loader = DocumentLoader(document_directory)
titles, documents = loader.load_documents()
    
huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
if not huggingface_api_key:
    raise ValueError("HUGGINGFACE_API_KEY not found in environment variables")

embeddings = Embeddings(model_id=model_id, HUGGINGFACE_API_KEY=huggingface_api_key)
document_embeddings, chunked_texts_with_titles = embeddings.get_embeddings(titles, documents, embedding_directory=embedding_directory)

In [ ]:
embedding_dimension = len(document_embeddings[0])
vector_store = VectorStore(dimension=embedding_dimension)
vector_store.add_documents(chunked_texts_with_titles, document_embeddings)

retriever = Retriever(vector_store, embeddings)
llm = LLM(config['llm']['api_url'])

rag_agent = RAGAgent(retriever=retriever, llm=llm)

In [ ]:
query = "When he felt a hand on his shoulder, who's hand was it and what did he say?"
prompt = rag_agent.get_prompt(query=query)
prompt

In [ ]:
answer = rag_agent.answer("When he felt a hand on his shoulder, who's hand was it and what did he say?")
answer